In [1]:
import pandas as pd

    1  English Premier League   1888-1889    Manchester United
0   2                 La Liga   1929-1930         Real Mardrid
1   3              Bundesliga   1903-1904        Bayern Munich
2   4                 Serie A   1898-1899             Juventus
3   5                 Ligue 1  1930 -1931  Paris Saint-Germain
4   6           Primeira Liga   1934-1935              Benfica
5   7  Russian Premier League   1992-1993       Spartak Moscow
6   8               Süper Lig   1959-1960          Galatasaray
7   9            Super League   1905-1906           Olympiacos
8  10              Eredivisie   1956-1957                 Ajax


In [3]:
df = pd.read_excel('./League.xlsx')
print(df)
df.to_csv('./League.csv', index=False)

    1  English Premier League   1888-1889    Manchester United
0   2                 La Liga   1929-1930         Real Mardrid
1   3              Bundesliga   1903-1904        Bayern Munich
2   4                 Serie A   1898-1899             Juventus
3   5                 Ligue 1  1930 -1931  Paris Saint-Germain
4   6           Primeira Liga   1934-1935              Benfica
5   7  Russian Premier League   1992-1993       Spartak Moscow
6   8               Süper Lig   1959-1960          Galatasaray
7   9            Super League   1905-1906           Olympiacos
8  10              Eredivisie   1956-1957                 Ajax


Crawl Data

Crawl Data

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

In [ ]:
standings_url = "https://fbref.com/en/comps/20/Bundesliga-Stats"
data = requests.get(standings_url) # data.text is html file
print(data)

<Response [200]>


In [ ]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [ ]:
standing = pd.read_html(data.text, attrs={'class': 'stats_table'})[0]

In [ ]:
team_urls = [f"https://fbref.com{l}" for l in links]
team_names = pd.read_excel('./passing.xlsx')
team_name = team_names['Bundesliga']
print(team_name)
print(team_urls)

0       Union Berlin
1      Bayern Munich
2           Dortmund
3         RB Leipzig
4     Eint Frankfurt
5           Freiburg
6          Wolfsburg
7         M'Gladbach
8         Leverkusen
9      Werder Bremen
10             KÃ¶ln
11          Mainz 05
12          Augsburg
13        Hoffenheim
14            Bochum
15         Stuttgart
16        Hertha BSC
17        Schalke 04
18               NaN
19               NaN
Name: Bundesliga, dtype: object
['https://fbref.com/en/squads/7a41008f/Union-Berlin-Stats', 'https://fbref.com/en/squads/054efa67/Bayern-Munich-Stats', 'https://fbref.com/en/squads/add600ae/Dortmund-Stats', 'https://fbref.com/en/squads/acbb6a5b/RB-Leipzig-Stats', 'https://fbref.com/en/squads/f0ac8ee6/Eintracht-Frankfurt-Stats', 'https://fbref.com/en/squads/a486e511/Freiburg-Stats', 'https://fbref.com/en/squads/4eaa11d7/Wolfsburg-Stats', 'https://fbref.com/en/squads/32f3ee20/Monchengladbach-Stats', 'https://fbref.com/en/squads/c7a9f859/Bayer-Leverkusen-Stats', 'https://fbref

In [ ]:
def clean(table, league_name):
    t = table[np.logical_and(table['Comp'] == league_name, table['Venue'] == 'Home')]
    t = t[['Venue', 'GF', 'GA', 'Opponent', 'Attendance', 'Referee']]
    t = t.dropna()
    t['GF'], t['GA'], t['Attendance'] = [t[col].astype(np.int32) for col in ['GF', 'GA', 'Attendance']]
    score = ['\'' + t.loc[r, 'GF'].astype(str) + ' - ' +  t.loc[r, 'GA'].astype(str) \
        if t.loc[r, 'Venue'] == 'Home' else '\''+ t.loc[r, 'GA'].astype(str) + ' - ' + t.loc[r, 'GF'].astype(str) \
        for r in t.index ]
    t.insert(4, 'Score', score, True)
    t = t.drop(columns=['GA', 'GF'])
    return t

In [ ]:
matches = pd.DataFrame(columns=['Team','Venue', 'Opponent', 'Score', 'Attendance', 'Referee'])
for i, (l, team) in enumerate(zip(team_urls, team_name)):
    if i == len(team_urls):
        break
    d = requests.get(l)
    match = clean(pd.read_html(d.text, match = "Scores & Fixtures")[0], 'Bundesliga')
    match.insert(0, 'Team', team, True)
    # print(match)
    # break
    matches = pd.concat([matches, match])
print(matches)

            Team Venue       Opponent   Score Attendance              Referee
1   Union Berlin  Home     Hertha BSC  '3 - 1      22012          Marco Fritz
3   Union Berlin  Home     RB Leipzig  '2 - 1      21056        Deniz Aytekin
5   Union Berlin  Home  Bayern Munich  '1 - 1      22012     Frank Willenborg
9   Union Berlin  Home      Wolfsburg  '2 - 0      22012      Robert Schröder
14  Union Berlin  Home       Dortmund  '2 - 0      22012       Tobias Stieler
..           ...   ...            ...     ...        ...                  ...
13    Schalke 04  Home       Freiburg  '0 - 2      61115    Christian Dingert
15    Schalke 04  Home       Mainz 05  '1 - 0      58256       Tobias Reichel
16    Schalke 04  Home  Bayern Munich  '0 - 2      62271         Felix Zwayer
18    Schalke 04  Home     RB Leipzig  '1 - 6      56841      Benjamin Cortus
19    Schalke 04  Home           Köln  '0 - 0      61571  Matthias Jöllenbeck

[166 rows x 6 columns]


In [ ]:
matches.to_csv('./matches4.csv')

In [ ]:
matches1 = pd.DataFrame(columns=['Team','Venue', 'Opponent', 'Score', 'Attendance', 'Referee'])

matches1 = pd.concat([matches1, pd.read_csv('./matches0.csv')])
matches1 = pd.concat([matches1, pd.read_csv('./matches1.csv')])
matches1 = pd.concat([matches1, pd.read_csv('./matches2.csv')])
matches1 = pd.concat([matches1, pd.read_csv('./matches3.csv')])
matches1 = pd.concat([matches1, pd.read_csv('./matches4.csv')])

print(matches1)
matches1.to_csv('./matches5.csv')

           Team Venue        Opponent   Score Attendance              Referee  \
0       Arsenal  Home  Leicester City  '4 - 2      60033       Darren England   
1       Arsenal  Home          Fulham  '2 - 1      60164       Jarred Gillett   
2       Arsenal  Home     Aston Villa  '2 - 1      60012         Robert Jones   
3       Arsenal  Home       Tottenham  '3 - 1      60278       Anthony Taylor   
4       Arsenal  Home       Liverpool  '3 - 2      60059       Michael Oliver   
..          ...   ...             ...     ...        ...                  ...   
161  Schalke 04  Home        Freiburg  '0 - 2      61115    Christian Dingert   
162  Schalke 04  Home        Mainz 05  '1 - 0      58256       Tobias Reichel   
163  Schalke 04  Home   Bayern Munich  '0 - 2      62271         Felix Zwayer   
164  Schalke 04  Home      RB Leipzig  '1 - 6      56841      Benjamin Cortus   
165  Schalke 04  Home            Köln  '0 - 0      61571  Matthias Jöllenbeck   

     Unnamed: 0  
0        

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

In [ ]:
standings_url = "https://fbref.com/en/comps/20/Bundesliga-Stats"
data = requests.get(standings_url) # data.text is html file
print(data)

<Response [200]>


In [ ]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [ ]:
standing = pd.read_html(data.text, attrs={'class': 'stats_table'})[0]

In [ ]:
team_urls = [f"https://fbref.com{l}" for l in links]
team_names = pd.read_excel('./passing.xlsx')
team_name = team_names['Bundesliga']
print(team_name)
print(team_urls)

0       Union Berlin
1      Bayern Munich
2           Dortmund
3         RB Leipzig
4     Eint Frankfurt
5           Freiburg
6          Wolfsburg
7         M'Gladbach
8         Leverkusen
9      Werder Bremen
10             KÃ¶ln
11          Mainz 05
12          Augsburg
13        Hoffenheim
14            Bochum
15         Stuttgart
16        Hertha BSC
17        Schalke 04
18               NaN
19               NaN
Name: Bundesliga, dtype: object
['https://fbref.com/en/squads/7a41008f/Union-Berlin-Stats', 'https://fbref.com/en/squads/054efa67/Bayern-Munich-Stats', 'https://fbref.com/en/squads/add600ae/Dortmund-Stats', 'https://fbref.com/en/squads/acbb6a5b/RB-Leipzig-Stats', 'https://fbref.com/en/squads/f0ac8ee6/Eintracht-Frankfurt-Stats', 'https://fbref.com/en/squads/a486e511/Freiburg-Stats', 'https://fbref.com/en/squads/4eaa11d7/Wolfsburg-Stats', 'https://fbref.com/en/squads/32f3ee20/Monchengladbach-Stats', 'https://fbref.com/en/squads/c7a9f859/Bayer-Leverkusen-Stats', 'https://fbref

In [ ]:
def clean(table, league_name):
    t = table[np.logical_and(table['Comp'] == league_name, table['Venue'] == 'Home')]
    t = t[['Venue', 'GF', 'GA', 'Opponent', 'Attendance', 'Referee']]
    t = t.dropna()
    t['GF'], t['GA'], t['Attendance'] = [t[col].astype(np.int32) for col in ['GF', 'GA', 'Attendance']]
    score = ['\'' + t.loc[r, 'GF'].astype(str) + ' - ' +  t.loc[r, 'GA'].astype(str) \
        if t.loc[r, 'Venue'] == 'Home' else '\''+ t.loc[r, 'GA'].astype(str) + ' - ' + t.loc[r, 'GF'].astype(str) \
        for r in t.index ]
    t.insert(4, 'Score', score, True)
    t = t.drop(columns=['GA', 'GF'])
    return t

In [ ]:
matches = pd.DataFrame(columns=['Team','Venue', 'Opponent', 'Score', 'Attendance', 'Referee'])
for i, (l, team) in enumerate(zip(team_urls, team_name)):
    if i == len(team_urls):
        break
    d = requests.get(l)
    match = clean(pd.read_html(d.text, match = "Scores & Fixtures")[0], 'Bundesliga')
    match.insert(0, 'Team', team, True)
    # print(match)
    # break
    matches = pd.concat([matches, match])
print(matches)

            Team Venue       Opponent   Score Attendance              Referee
1   Union Berlin  Home     Hertha BSC  '3 - 1      22012          Marco Fritz
3   Union Berlin  Home     RB Leipzig  '2 - 1      21056        Deniz Aytekin
5   Union Berlin  Home  Bayern Munich  '1 - 1      22012     Frank Willenborg
9   Union Berlin  Home      Wolfsburg  '2 - 0      22012      Robert Schröder
14  Union Berlin  Home       Dortmund  '2 - 0      22012       Tobias Stieler
..           ...   ...            ...     ...        ...                  ...
13    Schalke 04  Home       Freiburg  '0 - 2      61115    Christian Dingert
15    Schalke 04  Home       Mainz 05  '1 - 0      58256       Tobias Reichel
16    Schalke 04  Home  Bayern Munich  '0 - 2      62271         Felix Zwayer
18    Schalke 04  Home     RB Leipzig  '1 - 6      56841      Benjamin Cortus
19    Schalke 04  Home           Köln  '0 - 0      61571  Matthias Jöllenbeck

[166 rows x 6 columns]


In [ ]:
matches.to_csv('./matches4.csv')

In [ ]:
matches1 = pd.DataFrame(columns=['Team','Venue', 'Opponent', 'Score', 'Attendance', 'Referee'])

matches1 = pd.concat([matches1, pd.read_csv('./matches0.csv')])
matches1 = pd.concat([matches1, pd.read_csv('./matches1.csv')])
matches1 = pd.concat([matches1, pd.read_csv('./matches2.csv')])
matches1 = pd.concat([matches1, pd.read_csv('./matches3.csv')])
matches1 = pd.concat([matches1, pd.read_csv('./matches4.csv')])

print(matches1)
matches1.to_csv('./matches5.csv')

           Team Venue        Opponent   Score Attendance              Referee  \
0       Arsenal  Home  Leicester City  '4 - 2      60033       Darren England   
1       Arsenal  Home          Fulham  '2 - 1      60164       Jarred Gillett   
2       Arsenal  Home     Aston Villa  '2 - 1      60012         Robert Jones   
3       Arsenal  Home       Tottenham  '3 - 1      60278       Anthony Taylor   
4       Arsenal  Home       Liverpool  '3 - 2      60059       Michael Oliver   
..          ...   ...             ...     ...        ...                  ...   
161  Schalke 04  Home        Freiburg  '0 - 2      61115    Christian Dingert   
162  Schalke 04  Home        Mainz 05  '1 - 0      58256       Tobias Reichel   
163  Schalke 04  Home   Bayern Munich  '0 - 2      62271         Felix Zwayer   
164  Schalke 04  Home      RB Leipzig  '1 - 6      56841      Benjamin Cortus   
165  Schalke 04  Home            Köln  '0 - 0      61571  Matthias Jöllenbeck   

     Unnamed: 0  
0        